# AI-LAB SESSION 1: Uninformed search

In this first session we will work on uninformed search

## Maze environments

The environments used are **SmallMaze** (visible in the figure) and its variations
![SmallMaze](images/maze.png)
The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$

## Assignment 1

Your first assignment is to implement the BFS algorithm on **SmallMaze**. In particular, you are required to implement both *tree_search* and *graph_search* versions of BFS that will be called by the generic *bfs*.

The results returned by your *bfs* must be a tuple $(path, stats)$ in the following form:
* *path* - tuple of state identifiers forming a path from the start state to the goal state. ``None`` if no solution is found
* *stats* - tuple of:
     * *time* - time elapsed between the start and the end of the algorithm
     * *expc* - number of nodes explored. A node is considered as explored when removed from the fringe and analyzed
     * *maxnodes* - maximum number of nodes in memory at the same time (fringe + closed)

After the correctness of your implementations have been assessed, you can run the algorithms on other two maze environments: **GrdMaze** and **BlockedMaze**.

Function *build_path* can be used to return a tuple of states from the root node to another node by following *parent* links

In [5]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym
import envs
from timeit import default_timer as timer
from utils.fringe import FringeNode, QueueFringe

explored = 0 # var global 

def build_path(node):
    """
    Builds a path going backward from a node
    
    Args:
        node: node to start from
        
    Returns:
        path from root to ``node``
    """
    path = []
    while node.parent is not None:
        path.append(node.state)
        node = node.parent
    return tuple(reversed(path))

The next two functions have to be implemented

In [10]:
def tree_search(environment, fringe):
    """
    Tree search
    
    Args:
        environment: OpenAI Gym environment
        fringe: instance of Fringe data structure
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored states, max nodes in memory
    """
    
    maxNodes=0; explored=0;
    
    root = FringeNode(environment.startstate)
    
    fringe.add(root);
    
    while True:
        explored=explored+1
        if fringe.is_empty():
            return None, (explored,maxNodes);
        
        nodo = fringe.remove();
        
        if nodo.state==environment.goalstate:
            return build_path(nodo),(explored,maxNodes);
        
        #4 azioni possibili
        for action in range(env.action_space.n):
            #esploro i vicini
            child=environment.sample(nodo.state,action);
            child=FringeNode(child,nodo);
            
            #if child.state not in fringe:
            
            #N.B: commentando le 2 righe successive faccio il controllo
            #del goal state dopo e così viene come quello nelle soluzioni
            
            #if child.state == environment.goalstate:
             #   return build_path(child), (explored,maxNodes)
            fringe.add(child)
            maxNodes = max(maxNodes, len(fringe))

In [11]:
def graph_search(environment, fringe):
    """
    Graph search
    
    Args:
        environment: OpenAI Gym environment
        fringe: instance of Fringe data structure
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored nodes, max nodes in memory
    """
    
    maxNodes=0; explored=0;
    
    root = FringeNode(environment.startstate)
    
    fringe.add(root);
    
    closed=set();
    
    while True:
        
        if fringe.is_empty():
            return None, (explored,maxNodes);
        
        nodo = fringe.remove();
        explored=explored+1;

        if environment.goalstate == nodo.state:
            return build_path(nodo),(explored,maxNodes);
        
        closed.add(nodo.state)
        
        #4 azioni possibili
        for action in range(env.action_space.n):
            
            #esploro i vicini --sample() lavora solo sugli indici interi
            child=environment.sample(nodo.state,action);
            child=FringeNode(child,nodo);            
            
            if child.state not in fringe and child.state not in closed:
                if child.state == environment.goalstate:
                    return build_path(child), (explored,maxNodes)
                fringe.add(child)
            
            maxNodes=max(maxNodes, (len(fringe)+len(closed)))
                
                
                
                

In [14]:
def bfs(environment, search_type):
    """
    Breadth-first search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - tree_search or graph_search (function pointer)
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, stats = search_type(environment, QueueFringe())
    return path, (timer() - t, stats[0], stats[1])

The following code calls your tree search version of BFS and prints the results

In [15]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

envname2 = "GrdMaze-v0"

envname3 = "BlockedMaze-v0"

print("\n----------------------------------------------------------------")
print("\tTREE SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = bfs(env, tree_search)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	TREE SEARCH
	Environment:  SmallMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 21.3876s
N° of nodes explored: 103723
Max n° of nodes in memory: 311167
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for BFS tree search can be found [here](files/results/bfs_tree_search_results.txt)

The following code calls your graph search version of BFS and prints the results

In [16]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

envname2= "GrdMaze-v0"

envname3= "BlockedMaze-v0"

print("\n----------------------------------------------------------------")
print("\tGRAPH SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = bfs(env, graph_search)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	GRAPH SEARCH
	Environment:  SmallMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 0.0063s
N° of nodes explored: 15
Max n° of nodes in memory: 15
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for BFS graph search can be found [here](files/results/bfs_graph_search_results.txt)

## Assignment 2

Your second assignment is to implement the IDS algorithm on **SmallMaze**. In particular, you are required to implement both *dls_ts* (depth-limited tree search) and *dls_gs* (depth-limited graph search) versions of IDS that will be called by the generic *ids*. The recursions must be implemented in *rdls_ts* (recursive depth-limited tree search) and *rdls_gs* (recursive depth-limited graph search) called by *dls_ts* and *dls_gs* respectively.

Similarly to assignment 1, the results returned by your *ids* must be a tuple $(path, stats)$ in the following form:
* *path* - tuple of state identifiers forming a path from the start state to the goal state. ``None`` if no solution is found
* *stats* - tuple of:
     * *time* - time elapsed between the start and the end of the algorithm
     * *expc* - number of nodes explored. A node is considered as explored when removed from the fringe and analyzed
     * *maxnodes* - maximum number of nodes in memory at the same time (the depth of the recursion stack + closed)

After the correctness of your implementations have been assessed, you can run the algorithms on other two maze environments: **GrdMaze** and **BlockedMaze**.

**FringeNode** has a useful variable that can be set in the constructor and can be used to track the depth of a node in the path (and consequently of the recursion stack of IDS): *pathcost*. If the root node has a *pathcost* of 0, its children will have a *pathcost* increased by 1

In [7]:
start = env.startstate
root = FringeNode(start)  # parent = None and pathcost = 0 as default
child = FringeNode(env.sample(start, 0), root, root.pathcost + 1)  # pathcost is the third argument
print("Root pathcost: {}\tChild pathcost: {}".format(root.pathcost, child.pathcost))

Root pathcost: 0	Child pathcost: 1


Here you can implement the various functions requested

In [8]:
def dls_ts(environment, limit):
    """
    Depth-limited search (tree search)
    
    Args:
        environment: OpenAI Gym environment
        limit: depth limit budget
        
    Returns:
        (path, cutoff, stats): solution as a path, cutoff flag and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, cutoff, expc, maxdepth = rdls_ts(environment, FringeNode(environment.startstate), limit)
    return path, cutoff, (timer() - t, expc, maxdepth)

In [9]:
def rdls_ts(environment, node, limit):
    
    """
    Recursive depth-limited search (tree search version)
    
    Args:
        environment: OpenAI Gym environment
        node: node to explore
        limit: depth limit budget
    
    Returns:
        (path, cutoff, expc, maxdepth): path, cutoff flag, number of explored nodes, max nodes in memory
    """
    
    global explored;
    explored = explored + 1;
    
    maxdepth = node.pathcost
    
    if node.state == environment.goalstate:
        return build_path(node), False, explored, maxdepth
    
    if limit == 0:
        cutoff = True
        return None, cutoff, explored, maxdepth;
    
    cutoff_occurred = False;
    
    for action in range(environment.action_space.n):
  
        child = FringeNode(environment.sample(node.state, action), node, node.pathcost+1);
        
        path, cutoff, explored, new_depth = rdls_ts(environment, child, limit - 1);
        
        maxdepth = max(maxdepth, new_depth)
        
        if cutoff == True: #result == cutoff
            cutoff_occurred = True
        elif not(path == None and cutoff == False): #result != failure
            return path, cutoff, explored, maxdepth;
        
    if cutoff_occurred:
        cutoff = True
        return None, cutoff, explored, maxdepth
    return None, False, explored, maxdepth #return failure, so cutoff is false and result is none

    

In [10]:
def dls_gs(environment, limit):
    """
    Depth-limited search (graph search)
    
    Args:
        environment: OpenAI Gym environment
        limit: depth limit budget
        
    Returns:
        (path, cutoff, stats): solution as a path, cutoff flag and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    
    t = timer()
    path, cutoff, expc, maxdepth = rdls_gs(environment, FringeNode(environment.startstate), limit, closed=set())
    return path, cutoff, (timer() - t, expc, maxdepth);

In [11]:
def rdls_gs(environment, node, limit, closed):
    """
    Recursive depth-limited search (graph search version)
    
    Args:
        environment: OpenAI Gym environment
        node: node to explore
        limit: depth limit budget
        closed: completely explored nodes
        
    Returns:
        (path, cutoff, expc, maxdepth): path, cutoff flag, n° of nodes explored, max nodes in memory
    """
    
    global explored;
    maxdepth = node.pathcost;

    if node.state == environment.goalstate:
        explored += 1
        return (build_path(node), False, explored, maxdepth)
    
    if limit == 0:
        
        cutoff = True
        
        if node.state not in closed:
            explored += 1
            closed.add(node.state)
            
        return None, cutoff, explored, maxdepth
    
    cutoff_occurred = False
    
    if node.state not in closed:
        
        explored += 1
        closed.add(node.state)
        
        for action in range(environment.action_space.n):
    
            child = FringeNode(environment.sample(node.state, action), node, node.pathcost+1)
            path, cutoff, explored, new_depth = rdls_gs(environment, child, limit - 1, closed)
            
            maxdepth = max(maxdepth, new_depth)
            
            if cutoff == True: #result == cutoff
                cutoff_occurred = True
            elif not(path == None and cutoff == False): #result != failure
                return path, cutoff, explored, maxdepth
            
    if cutoff_occurred:
        cutoff = True
        return None, cutoff, explored, maxdepth
    return None, False, explored, maxdepth #return failure, so cutoff is false and result is none


In [12]:
def ids(environment, search_type):
    """
    Iterative deepening depth-first search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search (graph or tree) - dls_gs or dls_ts (function pointer)
    
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    
    # from depth = 0 to infinity
    #   solution, cutoff, stats = stype(problem, depth)
    # return solution, (timer() - t, expc, maxdepth)
    
    global explored
    explored = 0
    depth = 0

    t = timer()
    
    while True:
        solution, cutoff, stats = search_type(environment, depth)
        if cutoff is not True:
            return solution, (timer() - t, stats[1], stats[2] + 1)
        depth = depth + 1;

The following code calls your tree search version of IDS and prints the results

In [13]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

envname2 = "GrdMaze-v0"

envname3 = "BlockedMaze-v0"

print("\n----------------------------------------------------------------")
print("\tTREE SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = ids(env, dls_ts)  # Perform 
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	TREE SEARCH
	Environment:  SmallMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 15.9163s
N° of nodes explored: 138298
Max n° of nodes in memory: 10
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for IDS tree search can be found [here](files/results/ids_tree_search_results.txt)

The following code calls your graph search version of BFS and prints the results

In [16]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

envname2 = "GrdMaze-v0"

envname3 = "BlockedMaze-v0"

print("\n----------------------------------------------------------------")
print("\tGRAPH SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = ids(env, dls_gs)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	GRAPH SEARCH
	Environment:  SmallMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 0.0346s
N° of nodes explored: 118
Max n° of nodes in memory: 12
Solution: [(0, 1), (0, 0), (1, 0), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for IDS graph search can be found [here](files/results/ids_graph_search_results.txt)

## Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?

In [15]:
Metodologie graph search ottimizzano il numero di nodi esplorati

SyntaxError: invalid syntax (<ipython-input-15-734fcd5fb094>, line 1)